In [40]:
from bs4 import BeautifulSoup as BS
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import *
from sklearn.neighbors import NearestNeighbors
import time
import re
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV

from sklearn.metrics.pairwise import cosine_similarity

# Make DFs

In [2]:
df = pd.read_csv('updated_df.csv', index_col=0)

In [3]:
df.head()

,fit,user_id,bust_size,item_id,weight,rating,rented_for,review_text,body_type,review_summary,category,height,size,age,cup_size
0,fit,420272,34,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,14,28.0,d
1,fit,273551,34,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,12,36.0,b
3,fit,909926,34,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,8,34.0,c
4,fit,151944,34,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,12,27.0,b
5,fit,734848,32,364092,138lbs,8.0,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,68.0,8,45.0,b


In [4]:
#remove 'lbs'
df['weight'] = df['weight'].str.split('\D+')
df['weight'] = df['weight'].apply(lambda x:x[0])
df['weight'] = df['weight'].astype('int32')

In [5]:
df['user_id'].value_counts()

691468    436
32925     292
45337     140
833675    128
742630    124
         ... 
122743      1
329580      1
606059      1
821084      1
637431      1
Name: user_id, Length: 77237, dtype: int64

In [6]:
condition1 = df['user_id'] == 45337
condition2 = df['rented_for'] != 'party: cocktail'

In [7]:
dress_condition = ((df['category'] == 'dress') | (df['category'] == 'sheath') | (df['category'] == 'shirtdress') | 
(df['category'] == 'shift') | (df['category'] == 'ballgown') | (df['category'] == 'frock') | 
(df['category'] == 'kaftan') | (df['category'] == 'caftan') | (df['category'] == 'gown') | (df['category'] == 'print'))

In [8]:
dress_df = df[dress_condition]


In [9]:
df.category.value_counts()


dress           70342
gown            33240
sheath          14679
shift            4032
jumpsuit         4016
                ...  
sweatershirt        3
sweatpants          1
buttondown          1
crewneck            1
overcoat            1
Name: category, Length: 68, dtype: int64

In [10]:
#user df
user_df = df[['user_id', 'bust_size', 'cup_size', 'body_type', 'weight', 'height', 'age']].copy()


In [11]:
#df split into items only
items_df = df[['item_id', 'rating', 'rented_for', 'category']].copy()

In [12]:
#df filtered by item, dresses only
dress_items_df = dress_df[['item_id', 'rating', 'rented_for', 'category']].copy()

In [13]:
df['item_count'] = df.groupby(['item_id'])['item_id'].transform('count')

In [14]:
#df for subset of dress ratings
condition3 = df['item_count'] < 20

In [15]:
#how many items have been rented fewer than 20 times
len((df[df['item_count'] < 20])['item_id'].unique())

3876

In [16]:
#df of dresses with fewer than 20 rentals
small_df = df[condition3]
small_df.head()

,fit,user_id,bust_size,item_id,weight,rating,rented_for,review_text,body_type,review_summary,category,height,size,age,cup_size,item_count
0,fit,420272,34,2260466,137,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,14,28.0,d,19
6,fit,336066,34,568429,112,10.0,everyday,This dress is so sweet. I loved the print. The...,hourglass,LITERALLY THE CUTEST DRESS EVER,dress,63.0,4,27.0,c,3
20,fit,661150,36,900878,145,10.0,wedding,I rented this dress for a spring wedding. I ha...,hourglass,Perfect for a spring wedding!,dress,65.0,12,33.0,d,16
27,small,610914,32,1739627,140,10.0,party,"Lovely fabric, heavier than I imagined it woul...",hourglass,Runs small,dress,63.0,4,32.0,c,13
41,fit,606693,36,1542652,145,8.0,wedding,This dress is gorgeous and really accentuates ...,athletic,This dress definitely brought the WOW factor! ...,dress,68.0,12,27.0,b,10


In [17]:
small_df.shape

(28649, 16)

In [18]:
###one hot encode item df
dress_items_dummies = pd.get_dummies(dress_items_df)
dress_items_dummies.head()

,item_id,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,...,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
1,153475,10.0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,126335,8.0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,616682,10.0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
5,364092,8.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,568429,10.0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [19]:
#average rating for dresses only
items_df_dress = dress_items_dummies.groupby(['item_id']).mean()

In [59]:
items_df_dress.head()

,item_id,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,...,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
0,123373,8.871486,0.004016,0.000000,0.632530,0.036145,0.054217,0.0,0.002008,0.269076,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,123793,9.528804,0.005170,0.000000,0.618168,0.052437,0.041359,0.0,0.000000,0.281388,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,124204,9.326700,0.021559,0.000000,0.227197,0.111111,0.533997,0.0,0.029851,0.072968,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,124553,8.529248,0.094708,0.000000,0.172702,0.111421,0.300836,0.0,0.008357,0.275766,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,125424,9.025210,0.053221,0.005602,0.134454,0.201681,0.434174,0.0,0.005602,0.154062,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Normalize data

In [34]:
####log transform this DF before running cosine similarity
log_df = items_df_dress.transform(lambda x: np.log(x))
log_df.head()

/Users/brittanyfowle/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,item_id,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,...,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
0,11.722968,2.182842,-5.517453,-inf,-0.458027,-3.320228,-2.914763,-inf,-6.210600,-1.312760,...,-inf,-inf,-inf,-inf,0.0,-inf,-inf,-inf,-inf,-inf
1,11.726366,2.254319,-5.264908,-inf,-0.480994,-2.948139,-3.185467,-inf,-inf,-1.268019,...,-inf,-inf,-inf,-inf,0.0,-inf,-inf,-inf,-inf,-inf
2,11.729681,2.232881,-3.836968,-inf,-1.481936,-2.197225,-0.627366,-inf,-3.511545,-2.617728,...,-inf,-inf,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3,11.732487,2.143501,-2.356962,-inf,-1.756188,-2.194443,-1.201191,-inf,-4.784710,-1.288203,...,-inf,-inf,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf
4,11.739455,2.200022,-2.933297,-5.184589,-2.006535,-1.601070,-0.834311,-inf,-5.184589,-1.870403,...,-inf,-inf,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf


In [107]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dress_scaled = scaler.fit_transform(items_df_dress)
# Convert into a DataFrame
scaled_df = pd.DataFrame(dress_scaled, columns=items_df_dress.columns, index = items_df_dress.item_id)
scaled_df = scaled_df.drop(['item_id'], axis = 1)
scaled_df.head()

,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,rented_for_work,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
item_id,,,,,,,,,,,,,,,,,,,,
123373,0.858936,0.004016,0.000000,0.632530,0.036145,0.054217,0.0,0.002008,0.269076,0.002008,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
123793,0.941100,0.005170,0.000000,0.618168,0.052437,0.041359,0.0,0.000000,0.281388,0.001477,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
124204,0.915837,0.021559,0.000000,0.227197,0.111111,0.533997,0.0,0.029851,0.072968,0.003317,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124553,0.816156,0.094708,0.000000,0.172702,0.111421,0.300836,0.0,0.008357,0.275766,0.036212,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125424,0.878151,0.053221,0.005602,0.134454,0.201681,0.434174,0.0,0.005602,0.154062,0.011204,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# KNN

In [39]:
# from sklearn.neighbors import NearestNeighbors
# neigh = NearestNeighbors(n_neighbors= 5)
# neigh.fit(scaled_df)
# NearestNeighbors(n_neighbors=5)
# NN_Recs= neigh.kneighbors_graph(scaled_df)
# NN_Recs.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [80]:
#train test split
scaled_df.head()

,item_id,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,...,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
0,123373,0.858936,0.004016,0.000000,0.632530,0.036145,0.054217,0.0,0.002008,0.269076,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,123793,0.941100,0.005170,0.000000,0.618168,0.052437,0.041359,0.0,0.000000,0.281388,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,124204,0.915837,0.021559,0.000000,0.227197,0.111111,0.533997,0.0,0.029851,0.072968,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,124553,0.816156,0.094708,0.000000,0.172702,0.111421,0.300836,0.0,0.008357,0.275766,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,125424,0.878151,0.053221,0.005602,0.134454,0.201681,0.434174,0.0,0.005602,0.154062,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
#scaled_df = scaled_df.reset_index()

In [69]:
#scaled_df['rating'] = scaled_df['item_id'].astype(str)

In [82]:
rating = scaled_df['rating']

In [83]:
scaled_df.drop('rating', axis=1, inplace=True)

In [84]:
rating.dtypes

dtype('float64')

In [85]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, rating, test_size=0.25, random_state=42)

In [86]:
# Import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Instantiate KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit the classifier
clf.fit(X_train, y_train)

# Predict on the test set
test_preds = clf.predict(X_test)

ValueError: Unknown label type: 'continuous'

In [75]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [76]:
# Complete the function
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds)))
    print("Recall Score: {}".format(recall_score(labels, preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)))
    
print_metrics(y_test, test_preds)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [92]:
####pointless cosine sim
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dress_scaled = scaler.fit_transform(items_df_dress)
# Convert into a DataFrame
scaled_df = pd.DataFrame(dress_scaled, columns=items_df_dress.columns, index = items_df_dress.index)
scaled_df.head()
cos_sim = cosine_similarity(scaled_df)

In [90]:
scaled_df.head()

,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,rented_for_work,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
item_id,,,,,,,,,,,,,,,,,,,,
123373,0.858936,0.004016,0.000000,0.632530,0.036145,0.054217,0.0,0.002008,0.269076,0.002008,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
123793,0.941100,0.005170,0.000000,0.618168,0.052437,0.041359,0.0,0.000000,0.281388,0.001477,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
124204,0.915837,0.021559,0.000000,0.227197,0.111111,0.533997,0.0,0.029851,0.072968,0.003317,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124553,0.816156,0.094708,0.000000,0.172702,0.111421,0.300836,0.0,0.008357,0.275766,0.036212,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125424,0.878151,0.053221,0.005602,0.134454,0.201681,0.434174,0.0,0.005602,0.154062,0.011204,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
indices=pd.Series(scaled_df.index)

In [105]:
# scaled_df = scaled_df.set_index('item_id')

In [109]:
scaled_df.index[0]

123373

In [111]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(scaled_df.index)
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(item_id, cos_sim = cos_sim):
    
    # initializing the empty list of recommended movies
    rec_items = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == item_id].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        rec_items.append(list(scaled_df.index)[i])
        
    return rec_items

In [113]:
recommendations(616682)

[541999,
 698130,
 595269,
 712843,
 466944,
 503009,
 580060,
 737598,
 418998,
 755371]

# generate recs

In [155]:
def get_item_from_index(index):
    return items_df_dress[items_df_dress.index == index]['item_id'].values[0]

In [144]:
items_df_dress.head()

,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,rented_for_work,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
item_id,,,,,,,,,,,,,,,,,,,,
123373,8.871486,0.004016,0.000000,0.632530,0.036145,0.054217,0.0,0.002008,0.269076,0.002008,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
123793,9.528804,0.005170,0.000000,0.618168,0.052437,0.041359,0.0,0.000000,0.281388,0.001477,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
124204,9.326700,0.021559,0.000000,0.227197,0.111111,0.533997,0.0,0.029851,0.072968,0.003317,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124553,8.529248,0.094708,0.000000,0.172702,0.111421,0.300836,0.0,0.008357,0.275766,0.036212,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125424,9.025210,0.053221,0.005602,0.134454,0.201681,0.434174,0.0,0.005602,0.154062,0.011204,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
items_df_dress = items_df_dress.reset_index()

In [151]:
items_df_dress.head()

,item_id,rating,rented_for_date,rented_for_everyday,rented_for_formal affair,rented_for_other,rented_for_party,rented_for_party: cocktail,rented_for_vacation,rented_for_wedding,...,category_ballgown,category_caftan,category_dress,category_frock,category_gown,category_kaftan,category_print,category_sheath,category_shift,category_shirtdress
0,123373,8.871486,0.004016,0.000000,0.632530,0.036145,0.054217,0.0,0.002008,0.269076,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,123793,9.528804,0.005170,0.000000,0.618168,0.052437,0.041359,0.0,0.000000,0.281388,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,124204,9.326700,0.021559,0.000000,0.227197,0.111111,0.533997,0.0,0.029851,0.072968,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,124553,8.529248,0.094708,0.000000,0.172702,0.111421,0.300836,0.0,0.008357,0.275766,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,125424,9.025210,0.053221,0.005602,0.134454,0.201681,0.434174,0.0,0.005602,0.154062,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [152]:
def get_index_from_item(item_id):
     return items_df_dress[items_df_dress.item_id == item]['index'].values[0]

In [153]:
item_user_likes = 153475
item_rec = get_item_from_index(item_user_likes)
similar_items = list(enumerate(cosine_sim[items_df_dress])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it

IndexError: index 0 is out of bounds for axis 0 with size 0

In [55]:
dress_ratings_df = dress_df[['user_id', 'item_id', 'rating']]
dress_ratings_df.head()

,user_id,item_id,rating
1,273551,153475,10.0
3,909926,126335,8.0
4,151944,616682,10.0
5,734848,364092,8.0
6,336066,568429,10.0


# SVD

In [56]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(2.0, 10.0))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(dress_ratings_df, reader)

In [58]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3725  1.3916  1.3725  1.3580  1.3863  1.3762  0.0118  
MAE (testset)     1.0780  1.0847  1.0769  1.0699  1.0826  1.0784  0.0052  
Fit time          6.92    7.60    7.00    6.91    7.11    7.11    0.26    
Test time         0.35    0.35    0.18    0.18    0.19    0.25    0.08    


{'test_rmse': array([1.37246911, 1.3915513 , 1.37254066, 1.35797447, 1.38627628]),
 'test_mae': array([1.07796193, 1.0847293 , 1.07692066, 1.06985916, 1.08255057]),
 'fit_time': (6.924125909805298,
  7.604411840438843,
  6.995811939239502,
  6.906129837036133,
  7.106417894363403),
 'test_time': (0.35341715812683105,
  0.34746694564819336,
  0.18114709854125977,
  0.17920303344726562,
  0.18970227241516113)}

In [59]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

In [60]:
# rerun SVD on splits
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

# ##andrew's output: 
# RMSE: 1.4974
# Out[38]:
# 1.4974160703146928

RMSE: 1.3781


1.3780910216453208

# GridSearch

In [84]:
#original gridsearch
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

KeyboardInterrupt: 

In [ ]:
### grid search with updated params###

param_grid = {'n_epochs': [12,13], 'lr_all': [0.009, 0.01, 0.011],
              'reg_all': [0.35,0.4, 0.45]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

# KNN Basic

In [ ]:
condition4 = dress_df['item_count'] < 20
small_dress_df = dress_df[condition4]
dress_small_ratings_df = small_dress_df[['user_id', 'item_id', 'rating']]

In [ ]:
reader = Reader(rating_scale=(2,10))
data = Dataset.load_from_df(dress_small_ratings_df,reader)

In [34]:
# skipping
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1) #or small_df = data?
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))


#andrew's output
# ('test_rmse', array([1.49949173, 1.51515135, 1.52755969, 1.53797294, 1.537429  ]))
# ('test_mae', array([1.23950095, 1.25203504, 1.25227433, 1.25828419, 1.25664369]))
# ('fit_time', (20.733392000198364, 20.568089723587036, 20.11698579788208, 18.573551177978516, 5.321741104125977))
# ('test_time', (0.17890191078186035, 0.20891213417053223, 0.10373902320861816, 0.11112499237060547, 0.046755075454711914))
# -----------------------
# 1.5235209408682269


AttributeError: 'DataFrame' object has no attribute 'raw_ratings'

### KNN Baseline

In [ ]:
# cross validating with KNNBaseline
#data = ? i think data
from surprise.prediction_algorithms import KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline, data)
for i in cv_knn_baseline.items():
    print(i)
np.mean(cv_knn_baseline['test_rmse'])

# ###andrew's output

# Estimating biases using als...
# Computing the pearson similarity matrix...
# Done computing similarity matrix.
# Estimating biases using als...
# Computing the pearson similarity matrix...
# Done computing similarity matrix.
# Estimating biases using als...
# Computing the pearson similarity matrix...
# Done computing similarity matrix.
# Estimating biases using als...
# Computing the pearson similarity matrix...
# Done computing similarity matrix.
# Estimating biases using als...
# Computing the pearson similarity matrix...
# Done computing similarity matrix.
# ('test_rmse', array([1.49987156, 1.45807163, 1.46994754, 1.51157683, 1.48477858]))
# ('test_mae', array([1.20052122, 1.1659273 , 1.1856336 , 1.20520975, 1.18634309]))
# ('fit_time', (5.200239896774292, 5.399817943572998, 5.261378765106201, 5.1657209396362305, 5.300236225128174))
# ('test_time', (0.056257009506225586, 0.055994272232055664, 0.06219911575317383, 0.06447601318359375, 0.18319416046142578))
# Out[34]:
# 1.4848492272562706

# Get Recs

In [85]:
dress_big_ratings_df = dress_df[['user_id', 'item_id', 'rating']]


In [86]:
dress_big_ratings_df.head()


,user_id,item_id,rating
1,273551,153475,10.0
3,909926,126335,8.0
4,151944,616682,10.0
5,734848,364092,8.0
6,336066,568429,10.0


In [87]:
# index=user columns=item values=rating
dress_recs = pd.pivot_table(dress_big_ratings_df, values='rating', index='user_id', columns='item_id')

In [88]:
similar = dress_recs.corrwith(dress_recs[126335])
corr_similar = pd.DataFrame(similar,columns=['correlation'])

/Users/brittanyfowle/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/brittanyfowle/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [89]:
corr_similar.head()

,correlation
item_id,
123373,-0.454545
123793,NaN
124204,0.133631
124553,NaN
125424,0.707107


In [90]:
def get_recs(item, n_recs=5):
    similar = dress_recs.corrwith(dress_recs[item])
    corr_similar = pd.DataFrame(similar,columns=['correlation'])
    corr_similar.dropna(inplace=True)
    
    orig = dress_big_ratings_df.copy()
    
    corr_with_item = pd.merge(
            left=corr_similar,
            right=orig,
            on='item_id')[['item_id', 'correlation']].drop_duplicates().reset_index(drop=True)
    
    result = corr_with_item.sort_values(by='correlation', ascending=False)
    
    return result.head(n_recs)

In [91]:
get_recs(126335, n_recs=20)

,item_id,correlation
44,167112,1.0
17,135459,1.0
5,126335,1.0
101,1209909,1.0
27,144727,1.0
39,156607,1.0
29,146684,1.0
33,149655,1.0
78,651557,1.0
42,164593,1.0


In [92]:
dress_big_ratings_df['item_count'] = dress_big_ratings_df.groupby(['item_id'])['item_id'].transform('count')

/Users/brittanyfowle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:

dress_big_ratings_df['user_count'] = dress_big_ratings_df.groupby(['user_id'])['user_id'].transform('count')

/Users/brittanyfowle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
dress_big_ratings_df.shape

(123137, 5)

In [100]:
condition_u = dress_big_ratings_df['user_count'] > 5
condition_i = dress_big_ratings_df['item_count'] > 10

In [101]:
dress_ratings1 = dress_big_ratings_df[condition_u & condition_i]

In [102]:
len(dress_ratings1['item_id'].unique()) #why is mine more?
#andrew's output : 2018

2284

In [103]:
len(dress_ratings1['user_id'].unique()) #mine is more here too
#andrew's output : 465

2039

# Cosine Similarity

In [105]:
item_similarity = cosine_similarity(small_df)

ValueError: could not convert string to float: 'fit'